# RUN TEST SIMULATION

## Control run

* Prepare run directory: wps, wrf

* Namelist setting

* Run geogrid, ungrib, metgrid, real and wrf


In [1]:
import xarray as xr
import pandas as pd
import numpy as np
import os, sys, glob
import subprocess

### Preparing run directory


In [49]:
wpsdir              = "/Volumes/work/WRF_program/WRF_install/WPS/"
wrfdir              = "/Volumes/work/WRF_program/WRF_install/WRF_mpi/"
geogdir             = "/Volumes/work/WRF_program/WPS_GEOG"
renaldir            = "/Volumes/work/WRF_program/era5/"
namelist_wps_path   = "namelist/namelist.wps"
namelist_input_path = "namelist/namelist.input"

run_id              = 'tok_test_dom'
base_dir            = '/Users/doan/Google Drive/My Drive/share/2024/PGW_DS/'
run_dir             = base_dir + "/Run_WRF/"+run_id+"control"

In [4]:
st_date, en_date = "2023-01-01 00", "2023-01-01 18"

### Copy WPS program

In [5]:
import shutil
for idir in ['geogrid', 'ungrib', 'metgrid']: 
    # Copy the directory tree, allowing existing destination directory
    shutil.copytree(wpsdir+'/'+idir, run_dir+'/'+idir, dirs_exist_ok=True)
    shutil.copy2(wpsdir+'/'+idir+'.exe', run_dir+'/'+idir+'.exe')    
shutil.copy2(wpsdir+'/link_grib.csh', run_dir)

'/Users/doan/Google Drive/My Drive/share/2024/PGW_DS//Run_WRF/control/link_grib.csh'

## modify name list


In [6]:
import re
def modify_namelist(namelist_path_in, namelist_path_out, aa):
    with open(namelist_path_in, 'r') as file:
        lines = file.readlines()

    for i, line in enumerate(lines):
        for k, v in aa.items():
            parameter_pattern = re.compile(rf'^\s*{k}\s*=\s*')
            if parameter_pattern.match(line):
                lines[i] = f' {k} = {v},\n'

    with open(namelist_path_out, 'w') as file:
        file.writelines(lines)

In [7]:
namelist_path_in = namelist_wps_path
namelist_path_out = run_dir+'/namelist.wps'
aa = {'geog_data_path': '"'+geogdir+'"' }

modify_namelist(namelist_path_in, namelist_path_out, aa)


### Run geogrid.exe

In [ ]:
subprocess.run(['./geogrid.exe'], cwd=run_dir, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL, check=True)

### Run ungrib

In [13]:
source_file = os.path.join(run_dir, 'ungrib/Variable_Tables/Vtable.ECMWF')
target_link = os.path.join(run_dir, 'Vtable')
original_dir = os.getcwd()
os.chdir(run_dir)
if os.path.islink(target_link) or os.path.exists(target_link): os.remove(target_link)
os.symlink(source_file, target_link) # link
os.chdir(original_dir)

Symbolic link created from /Users/doan/Google Drive/My Drive/share/2024/PGW_DS//Run_WRF/control/ungrib/Variable_Tables/Vtable.ECMWF to /Users/doan/Google Drive/My Drive/share/2024/PGW_DS//Run_WRF/control/Vtable
Moved back to the original directory: /Users/doan/Documents/GitHub/Pseudo-Global-Warming-Downscaling


In [42]:
namelist_wps = run_dir + '/namelist.wps'
prefixes = ['ERA5A', 'ERA5S']
for i in range(2):
    prefix = prefixes[i]
    # Define the appropriate reanalysis file based on 'pressure' or 'surface' level
    reanal_files = renaldir + 'era5_ungrib_' + ['pressure', 'surface'][i] + '_levels_20230101.grib'
    
    subprocess.run(['./link_grib.csh', reanal_files], cwd=run_dir, check=True)
    subprocess.run(['rm', '-f'] + glob.glob(os.path.join(run_dir, prefix+'*')), check=True)

    modify_namelist(namelist_wps, namelist_wps, {'prefix': f'"{prefix}"'})
    subprocess.run(['./ungrib.exe'], cwd=run_dir, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL, check=True)

In [29]:
#os.system('cd '+run_dir+';ln -sf ungrib/Variable_Tables/Vtable.ECMWF Vtable')

0

### Run metgrid

In [43]:
fg_name = '"'+'","'.join(prefixes)+'"'
modify_namelist(namelist_wps, namelist_wps, { 'fg_name'  : fg_name} )

In [44]:
subprocess.run(['./metgrid.exe'], cwd=run_dir, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL, check=True)

In [34]:
#os.system('cd '+run_dir+'; ./metgrid.exe  > NUL 2>&1; cd -')

/Users/doan/Documents/GitHub/Pseudo-Global-Warming-Downscaling/03_WRF_run


0

## Copy WRF run files

In [45]:
source_dir = os.path.join(wrfdir, 'run')
destination_dir = run_dir

for file_name in os.listdir(source_dir):
    full_file_name = os.path.join(source_dir, file_name)    
    if os.path.isfile(full_file_name):
        shutil.copy(full_file_name, destination_dir)

### Copy namelist to destination

In [50]:
with open(namelist_input_path, 'r') as file: a1 = file.readlines()
with open(run_dir+'/namelist.input', 'w') as file: file.writelines(a1)

In [51]:
subprocess.run(['./real.exe'], cwd=run_dir, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL, check=True)

In [52]:
command = ['mpirun', '-np', '4', './wrf.exe']
subprocess.run(command, cwd=run_dir, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL, check=True)


## Copy all directory to PGW






In [ ]:
src_dir             = base_dir + "/Run_WRF/"+  run_id  + "/control"
dst_dir             = base_dir + "/Run_WRF/" + run_id  + "/PGW_DS"

In [ ]:
import os
import shutil
import fnmatch

def copy_files_except(src, dst, exclude_patterns):
    # Ensure the destination directory exists
    if not os.path.exists(dst): os.makedirs(dst)

    # Iterate over all items in the source directory
    for item in os.listdir(src):
        s = os.path.join(src, item)
        d = os.path.join(dst, item)

        # Check if the file matches any of the exclude patterns
        if os.path.isfile(s) and not any(fnmatch.fnmatch(item, pattern) for pattern in exclude_patterns):
            shutil.copy2(s, d)

# Patterns to exclude (e.g., *.txt files and files starting with 'temp')
exclude_patterns = ['met_em*.nc', 'wrfout_*.nc', '*:*']
copy_files_except(src_dir, dst_dir, exclude_patterns)

shutil.copytree(src_dir+'/metgrid', dst_dir+'/metgrid', dirs_exist_ok=True)